In [3]:
# Global UUID is generated for agent and task
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4() 

### Creating Julep Client with the API Key

Get your API key from [here](https://dashboard.julep.ai/)

In [4]:
from julep import Client
import os
from dotenv import load_dotenv

load_dotenv()

JULEP_API_KEY = os.environ["JULEP_API_KEY"]

# Create a Julep client
client = Client(api_key=JULEP_API_KEY, environment="production")

In [5]:
# Defining the agent
name = "Jacob"
about = "A travel assistant that helps plan the perfect trip."

# Create the agent
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name=name,
    about=about,
    model="gpt-4o",
)

In [7]:
import yaml

openweathermap_api_key =  os.environ['OPEN_WEATHER']
brave_api_key = os.environ['BRAVE']

# Defining the task
task_def = yaml.safe_load(f"""
# yaml-language-server: $schema=https://raw.githubusercontent.com/julep-ai/julep/refs/heads/dev/schemas/create_task_request.json
name: Julep Tourist Plan With Weather And Attractions
description: A task that plans a trip with weather and attractions.

########################################################
####################### INPUT SCHEMA ##################
########################################################                       
input_schema:
  type: object
  properties:
    locations:
      type: array
      items:
        type: string
      description: The locations to search for.

########################################################
####################### TOOLS ##########################
########################################################

# Define the tools that the task will use in this workflow
tools:
- name: wikipedia
  type: integration
  integration:
    provider: wikipedia

- name: weather
  type: integration
  integration:
    provider: weather
    setup:
      openweathermap_api_key: {openweathermap_api_key}

- name: internet_search
  type: integration
  integration:
    provider: brave
    setup:
      brave_api_key: {brave_api_key}

########################################################
####################### MAIN WORKFLOW ##########################
########################################################

main:
# Step 0: Fetch weather data for each location
- over: $ steps[0].input.locations
  map:
    tool: weather
    arguments:
      location: $ _

# Step 1: Search Wikipedia for tourist attractions for each location
- over: $ steps[0].input.locations
  map:
    tool: internet_search
    arguments:
      query: $ 'tourist attractions in ' + _

# Step 2: Zip locations, weather, and attractions into a list of tuples [(location, weather, attractions)]
- evaluate:
    zipped: |-
      $ list(
        zip(
          steps[0].input.locations,
          [output['result'] for output in steps[0].output],
          steps[1].output
        )
      )

# Step 3: Create an itinerary for each location
- over: $ _['zipped']
  parallelism: 3
  # Inside the map step, each `_` represents the current element in the list
  # which is a tuple of (location, weather, attractions)
  map:
    prompt:
    - role: system
      content: >-
        $ f'''You are {{agent.name}}. Your task is to create a detailed itinerary
        for visiting tourist attractions in some locations.
        The user will give you the following information for each location:

        - The location
        - The current weather condition
        - The top tourist attractions'''
    - role: user
      content: >-
        $ f'''Location: "{{_[0]}}"
        Weather: "{{_[1]}}"
        Attractions: "{{_[2]}}"'''
    unwrap: true

# Step 4: Create a final plan by joining the activities for each location
- evaluate:
    final_plan: |-
      $ '\\n---------------\\n'.join(activity for activity in _)
""")

In [8]:
# creating the task object
task = client.tasks.create_or_update(
    task_id=TASK_UUID,
    agent_id=AGENT_UUID,
    **task_def
)

### Creating an Execution


In [9]:
execution = client.executions.create(
    task_id=task.id,
    input={
         "locations": ["New York", "London", "Paris", "Tokyo", "Sydney"]
    }
)

print("Started an execution. Execution ID:", execution.id)

Started an execution. Execution ID: 068016b1-3f05-7c23-8000-2e45cb33526d


## Checking execution details and output

In [10]:
import time

execution = client.executions.get(execution.id)

while execution.status != "succeeded":
    time.sleep(5)
    execution = client.executions.get(execution.id)
    print("Execution status: ", execution.status)
    print("-"*50)

execution = client.executions.get(execution.id)

if 'final_plan' in execution.output:
    print(execution.output['final_plan'])
else:
    print(execution.output)


Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
--------------------------------------------------
Execution status:  running
-------------------------------------

In [49]:
# Lists all the task steps that have been executed up to this point in time
transitions = client.executions.transitions.list(execution_id=execution.id).items

# Transitions are retrieved in reverse chronological order
for transition in reversed(transitions):
    print("Transition type: ", transition.type)
    print("Transition output: ", transition.output)
    print("-"*50)

Transition type:  init
Transition output:  {'locations': ['New York', 'London', 'Paris', 'Tokyo', 'Sydney']}
--------------------------------------------------
Transition type:  init_branch
Transition output:  Sydney
--------------------------------------------------
Transition type:  init_branch
Transition output:  London
--------------------------------------------------
Transition type:  init_branch
Transition output:  Tokyo
--------------------------------------------------
Transition type:  init_branch
Transition output:  Paris
--------------------------------------------------
Transition type:  init_branch
Transition output:  New York
--------------------------------------------------
Transition type:  finish_branch
Transition output:  {'result': 'In London, the current weather is as follows:\nDetailed status: scattered clouds\nWind speed: 8.05 m/s, direction: 222°\nHumidity: 45%\nTemperature: \n  - Current: 14.87°C\n  - High: 15.83°C\n  - Low: 13.86°C\n  - Feels like: 13.59°C\nR